# Exercise 13

This particular Automobile Data Set includes a good mix of categorical values as well as continuous values and serves as a useful example that is relatively easy to understand. Since domain understanding is an important aspect when deciding how to encode various categorical values - this data set makes a good case study.

Read the data into Pandas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

# Define the headers since the data does not have any
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

# Read in the CSV file and convert "?" to NaN
df = pd.read_csv("http://mlr.cs.umass.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )
df = pd.DataFrame(df)
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [2]:
df.shape

(205, 26)

In [3]:
df.dtypes

symboling              int64
normalized_losses    float64
make                  object
fuel_type             object
aspiration            object
num_doors             object
body_style            object
drive_wheels          object
engine_location       object
wheel_base           float64
length               float64
width                float64
height               float64
curb_weight            int64
engine_type           object
num_cylinders         object
engine_size            int64
fuel_system           object
bore                 float64
stroke               float64
compression_ratio    float64
horsepower           float64
peak_rpm             float64
city_mpg               int64
highway_mpg            int64
price                float64
dtype: object

In [4]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,engine_type,num_cylinders,fuel_system
0,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
1,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
2,alfa-romero,gas,std,two,hatchback,rwd,front,ohcv,six,mpfi
3,audi,gas,std,four,sedan,fwd,front,ohc,four,mpfi
4,audi,gas,std,four,sedan,4wd,front,ohc,five,mpfi


**Se eliminan los registros que no tienen Price, porque esta es la variable a predecir con el modelo**

In [5]:
df.loc[df.price.isnull() == True]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
9,0,NaN,audi,gas,turbo,two,hatchback,4wd,front,99.5,...,131,mpfi,3.13,3.40,7.0,160.0,5500.0,16,22,NaN
44,1,NaN,isuzu,gas,std,two,sedan,fwd,front,94.5,...,90,2bbl,3.03,3.11,9.6,70.0,5400.0,38,43,NaN
45,0,NaN,isuzu,gas,std,four,sedan,fwd,front,94.5,...,90,2bbl,3.03,3.11,9.6,70.0,5400.0,38,43,NaN
129,1,NaN,porsche,gas,std,two,hatchback,rwd,front,98.4,...,203,mpfi,3.94,3.11,10.0,288.0,5750.0,17,28,NaN


In [6]:
df.dropna(subset=['price'],inplace = True)

In [7]:
df.isna().sum()

symboling             0
normalized_losses    37
make                  0
fuel_type             0
aspiration            0
num_doors             2
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            2
peak_rpm              2
city_mpg              0
highway_mpg           0
price                 0
dtype: int64

In [8]:
df.shape

(201, 26)

**Se modifica la variable num_doors de string a int, y se reemplazan los datos faltantes con la moda**

In [9]:
df.num_doors.unique()

array(['two', 'four', nan], dtype=object)

In [10]:
df.num_doors.value_counts()

four    113
two      86
Name: num_doors, dtype: int64

In [11]:
df.num_doors.fillna('four', inplace=True)

In [12]:
df.isna().sum()

symboling             0
normalized_losses    37
make                  0
fuel_type             0
aspiration            0
num_doors             0
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            2
peak_rpm              2
city_mpg              0
highway_mpg           0
price                 0
dtype: int64

In [13]:
df.loc[df.horsepower.isnull() == True].num_cylinders

130    four
131    four
Name: num_cylinders, dtype: object

In [14]:
df.groupby('num_cylinders')['horsepower'].value_counts().to_frame()

horsepower
num_cylinders horsepower            
eight         155.0                2
              184.0                2
five          110.0                4
              123.0                4
              115.0                1
              140.0                1
four          68.0                19
              69.0                10
              70.0                 9
              116.0                9
              95.0                 7
              62.0                 6
              88.0                 6
              114.0                6
              76.0                 5
              82.0                 5
              84.0                 5
              97.0                 5
              102.0                5
              145.0                5
              86.0                 4
              92.0                 4
              110.0                4
              111.0                4
              73.0                 3
              85.0                 3
              90.0                 3
              101.0                3
              160.0                3
              52.0                 2
...                              ...
              94.0                 2
              100.0                2
              112.0                2
              162.0                2
              55.0                 1
              58.0                 1
              60.0                 1
              64.0                 1
              72.0                 1
              78.0                 1
              120.0                1
              142.0                1
              143.0                1
              175.0                1
six           121.0                3
              152.0                3
              182.0                3
              207.0                3
              156.0                2
              160.0                2
              161.0                2
              176.0                2
              106.0                1
              134.0                1
              154.0                1
              200.0                1
three         48.0                 1
twelve        262.0                1
two           101.0                3
              135.0                1

[61 rows x 1 columns]

**Reemplazamos los datos faltantes con la moda de *horsepower* para *num_cylinders*='four'** 

In [15]:
df.horsepower.fillna(68, inplace=True)

In [16]:
df.loc[df.horsepower.isnull() == True]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price


In [17]:
df.loc[df.peak_rpm.isnull() == True].num_cylinders

130    four
131    four
Name: num_cylinders, dtype: object

In [18]:
df.groupby('num_cylinders')['peak_rpm'].value_counts().to_frame()

peak_rpm
num_cylinders peak_rpm          
eight         4500.0           2
              4750.0           2
five          5500.0           6
              4350.0           4
four          4800.0          35
              5500.0          29
              5000.0          25
              5200.0          13
              5400.0           8
              5250.0           7
              5800.0           7
              4150.0           5
              4200.0           5
              4500.0           5
              6000.0           5
              4400.0           3
              5100.0           2
              6600.0           2
              4650.0           1
              4900.0           1
              5300.0           1
              5600.0           1
six           5200.0          10
              4250.0           3
              5400.0           3
              5900.0           3
              4750.0           2
              4800.0           1
              5000.0           1
              5500.0           1
three         5100.0           1
twelve        5000.0           1
two           6000.0           4

**Reemplazamos los datos faltantes con la moda de *peak_rpm* para *num_cylinders*='four'** 

In [19]:
df.peak_rpm.fillna(4800, inplace=True)

In [20]:
df.loc[df.peak_rpm.isnull() == True]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price


In [21]:
df.isna().sum()

symboling             0
normalized_losses    37
make                  0
fuel_type             0
aspiration            0
num_doors             0
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            0
peak_rpm              0
city_mpg              0
highway_mpg           0
price                 0
dtype: int64

In [22]:
df.loc[df.bore.isnull() == True]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
55,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,NaN,NaN,9.4,101.0,6000.0,17,23,10945.0
56,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,NaN,NaN,9.4,101.0,6000.0,17,23,11845.0
57,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,NaN,NaN,9.4,101.0,6000.0,17,23,13645.0
58,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,80,mpfi,NaN,NaN,9.4,135.0,6000.0,16,23,15645.0


In [23]:
df.groupby(['make','drive_wheels'])['bore'].mean().to_frame()

bore
make          drive_wheels          
alfa-romero   rwd           3.206667
audi          4wd           3.190000
              fwd           3.178000
bmw           rwd           3.473750
chevrolet     fwd           2.990000
dodge         fwd           3.094444
honda         fwd           3.021538
isuzu         rwd           3.370000
jaguar        rwd           3.600000
mazda         fwd           3.230909
              rwd           3.595000
mercedes-benz rwd           3.605000
mercury       rwd           3.780000
mitsubishi    fwd           3.250769
nissan        fwd           3.219333
              rwd           3.430000
peugot        rwd           3.582727
plymouth      fwd           3.043333
              rwd           3.590000
porsche       rwd           3.790000
renault       fwd           3.460000
saab          fwd           3.373333
subaru        4wd           3.620000
              fwd           3.620000
toyota        4wd           3.050000
              fwd           3.200000
              rwd           3.404286
volkswagen    fwd           3.130000
volvo         rwd           3.662727

**Reemplazamos los valores faltantes con el promedio de 'drive_wheels == rwd' para 'make == mazda'**

In [24]:
df.bore.fillna(3.595, inplace=True)

In [25]:
df.loc[df.bore.isnull() == True]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price


In [26]:
df.loc[df.stroke.isnull() == True]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
55,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,3.595,NaN,9.4,101.0,6000.0,17,23,10945.0
56,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,3.595,NaN,9.4,101.0,6000.0,17,23,11845.0
57,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,3.595,NaN,9.4,101.0,6000.0,17,23,13645.0
58,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,80,mpfi,3.595,NaN,9.4,135.0,6000.0,16,23,15645.0


In [27]:
df.groupby(['make','drive_wheels'])['stroke'].mean().to_frame()

stroke
make          drive_wheels          
alfa-romero   rwd           2.943333
audi          4wd           3.400000
              fwd           3.400000
bmw           rwd           3.167500
chevrolet     fwd           3.083333
dodge         fwd           3.365556
honda         fwd           3.462308
isuzu         rwd           3.230000
jaguar        rwd           3.700000
mazda         fwd           3.280909
              rwd           3.400000
mercedes-benz rwd           3.432500
mercury       rwd           3.120000
mitsubishi    fwd           3.493846
nissan        fwd           3.322000
              rwd           3.270000
peugot        rwd           3.160000
plymouth      fwd           3.295000
              rwd           3.860000
porsche       rwd           2.952500
renault       fwd           3.900000
saab          fwd           2.903333
subaru        4wd           2.640000
              fwd           2.600000
toyota        4wd           3.030000
              fwd           3.217500
              rwd           3.330000
volkswagen    fwd           3.400000
volvo         rwd           3.147273

**Reemplazamos los valores faltantes con el promedio de 'drive_wheels == rwd' para 'make == mazda'**

In [28]:
df.stroke.fillna(3.4, inplace=True)

In [29]:
df.loc[df.bore.isnull() == True]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price


In [30]:
df.isna().sum()

symboling             0
normalized_losses    37
make                  0
fuel_type             0
aspiration            0
num_doors             0
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  0
stroke                0
compression_ratio     0
horsepower            0
peak_rpm              0
city_mpg              0
highway_mpg           0
price                 0
dtype: int64

# Exercise 13.1

Does the database contain missing values? If so, replace them using one of the methods explained in class

In [31]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

**Creamos dos dataframe uno con los na y otro sin los na**

In [32]:
df_nona = df.iloc[:].dropna()

In [33]:
df_na = df[pd.isna(df['normalized_losses'])]

**Creamos X e Y para correr el modelo**

In [34]:
X = df_nona.drop(['normalized_losses'], axis=1)
y = df_nona.normalized_losses.dropna()

In [35]:
import category_encoders as ce
BinEncoder = ce.BinaryEncoder().fit(X)
X_ = BinEncoder.transform(X)

In [36]:
rf.fit(X_,y)

C:\Users\Cristian Najera Leal\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

**Tomamos los registros a predecir**

In [37]:
y_test = df_na.drop(['normalized_losses'], axis=1)
y_test_ = BinEncoder.transform(y_test)

**Se crea un nuevo dataframe con las predicciones realizadas y se pone el index de los registros faltantes**

In [38]:
df_norm_pred = pd.DataFrame(rf.predict(y_test_), columns=['normalized_losses'], index=y_test_.index)
df_norm_pred.head()

,normalized_losses
0,171.2
1,172.8
2,153.3
5,142.6
7,158.6


**Se aplica fillna con las predicciones (por eso es importante poner bien el index arriba) y se verifica si hay o no na values**

In [39]:
df.normalized_losses.fillna(df_norm_pred.normalized_losses, inplace=True)

In [40]:
df.isna().sum()

symboling            0
normalized_losses    0
make                 0
fuel_type            0
aspiration           0
num_doors            0
body_style           0
drive_wheels         0
engine_location      0
wheel_base           0
length               0
width                0
height               0
curb_weight          0
engine_type          0
num_cylinders        0
engine_size          0
fuel_system          0
bore                 0
stroke               0
compression_ratio    0
horsepower           0
peak_rpm             0
city_mpg             0
highway_mpg          0
price                0
dtype: int64

# Exercise 13.2

Split the data into training and testing sets

Train a Random Forest Regressor to predict the price of a car using the nominal features. No son las nominales sino las escalares

In [41]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
num_df.head()

,symboling,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,171.2,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,172.8,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,153.3,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [42]:
X2 = num_df.drop(['price'], axis=1)
y2 = num_df.price.dropna()

In [43]:
(((pd.Series(cross_val_score(RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42), X2, y2, cv=100, scoring='neg_mean_squared_error')).describe())*-1)**(0.5))['mean']

2290.026460383906

# Exercise 13.3

Create dummy variables for the categorical features

Train a Random Forest Regressor and compare

In [44]:
df_dummies = pd.get_dummies(df)

In [45]:
X3 = df_dummies.drop(['price'], axis=1)
y3 = df_dummies.price

In [46]:
(((pd.Series(cross_val_score(RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42), X3, y3, cv=100, scoring='neg_mean_squared_error')).describe())*-1)**(0.5))['mean']

2296.602297998683

# Exercise 13.4

Apply two other methods of categorical encoding

compare the results

**Binary**

In [47]:
X4 = df.drop(['price'], axis=1)
y4 = df.price

In [48]:
BinEncoder = ce.BinaryEncoder().fit(X4)
X4_ = BinEncoder.transform(X4)

In [49]:
(((pd.Series(cross_val_score(RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42), X4_, y4, cv=100, scoring='neg_mean_squared_error')).describe())*-1)**(0.5))['mean']

2264.458611802847

**Feature Hashing Encoder**

In [52]:
rm = []
for i in range(5,51,5):
    X6 = ce.HashingEncoder(n_components=i).fit_transform(df.drop(['price'], axis=1))
    rm.append([i, (((pd.Series(cross_val_score(RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42), X6, y4, cv=100, scoring='neg_mean_squared_error')).describe())*-1)**(0.5))['mean']])
    print(i)

5
10
15
20
25
30
35
40
45
50


In [53]:
rm = pd.DataFrame(rm, columns=['Cant_Col', 'RMSE'])

In [54]:
rm.iloc[rm.RMSE.idxmin()]

Cant_Col      50.000000
RMSE        2257.062893
Name: 9, dtype: float64

**Ordinal**

In [55]:
X7 = ce.OrdinalEncoder().fit_transform(df.drop(['price'], axis=1))

In [56]:
(((pd.Series(cross_val_score(RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42), X7, y4, cv=100, scoring='neg_mean_squared_error')).describe())*-1)**(0.5))['mean']

2254.491719190035